In [1]:
import os
import json
import pickle
from data_loader.data_configs import abbre2language, language2abbre
import pandas as pd
import numpy as np

In [2]:
import pickle

def save_pickle(file, data):
    with open(file, 'wb') as f:
        pickle.dump(data, f)
        
def load_pickle(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [43]:
for dataset in ['marc', 'pawsx', 'xnli', 'panx', 'udpos']:
    if dataset == 'marc':
        shots = [1, 2, 4, 8]
        langs = ['chinese', 'french', 'german', 'japanese', 'spanish']
    elif dataset == 'pawsx':
        shots = [1, 2, 4, 8]
        langs = ['chinese', 'french', 'german', 'japanese', 'korean', 'spanish']
    elif dataset == 'xnli':
        shots = [1, 2, 4, 8]
        langs = ['arabic', 'bulgarian', 'chinese', 'french', 'german', 'greek', 'hindi', 'russian', 'spanish', 'swahili', 'thai', 'turkish', 'urdu', 'vietnamese']
    elif dataset == 'panx':
        shots = [1, 2, 4]
        langs = ['af', 'ar', 'bg', 'bn', 'de', 'el', 'en', 'es', 'et', 'eu', 'fa', 'fi', 'fr', 'he', 'hi', 'hu', 'id', 'it', 'ja', 'jv', 'ka', 'kk', 'ko', 'ml', 'mr', 'ms', 'my', 'nl', 'pt', 'ru', 'sw', 'ta', 'te', 'th', 'tl', 'tr', 'ur', 'vi', 'yo', 'zh']
        continue
    elif dataset == 'udpos':
        shots = [1, 2, 4]
        langs = ['af', 'ar', 'bg', 'de', 'el', 'en', 'es', 'et', 'eu', 'fa', 'fi', 'fr', 'he', 'hi', 'hu', 'id', 'it', 'ja', 'ko', 'mr', 'nl', 'pt', 'ru', 'ta', 'te', 'tr', 'ur', 'vi', 'zh']
        continue
    else:
        raise ValueError
        
    bucket_dict = {}
    for shot in shots:
        for lang in langs:
            for group_index in range(40):
                json_filename = './buckets/{}/{}-shots/{}/{}-th/sampled.json'.format(dataset, shot, lang, group_index)
                with open(json_filename, "r") as f:
                    json_file = json.load(f)
                    
                for _, v in json_file.items():
                    key, idx, _ = v[0]['uid'].split('-')
                    if key not in bucket_dict.keys():
                        bucket_dict[key] = [int(idx)]
                    else:
                        bucket_dict[key].append(int(idx))
                        
    os.makedirs('./buckets/index', exist_ok=True)
    save_pickle('./buckets/index/{}.pkl'.format(dataset), bucket_dict)
    
    os.makedirs('./buckets/left_dev/{}'.format(dataset), exist_ok=True)
    for lang in langs:
        if dataset in ['pawsx', 'xnli']:
            dev = pd.read_csv('./data/download/{}/dev-{}.tsv'.format(dataset, language2abbre[lang]),
                              sep='\t', header=None)
        else:
            continue

        idxs = load_pickle('./buckets/index/{}.pkl'.format(dataset))[lang]
        left_dev = []
        for i in range(dev.count()[0]):
            if i+1 not in idxs:
                left_dev.append(np.array(dev.iloc[i]).tolist())
        left_dev = pd.DataFrame(left_dev)
        left_dev.to_csv('./buckets/left_dev/{}/{}.tsv'.format(dataset, language2abbre[lang]),
                        sep='\t', header=None, index=False)